Dataset Loading

### Import Modules and Verification

In [1]:
import torch
print("PyTorch - CUDA available:", torch.cuda.is_available())

import xgboost as xgb

# Create a simple DMatrix
dtrain = xgb.DMatrix(data=[[1, 2], [3, 4], [5, 6]], label=[0, 1, 0])

# Train a small model with GPU support
params = {
 'tree_method': 'hist',  # Use the updated method
        'device': 'cuda',       # Specify GPU usage
}

# Train the model
bst = xgb.train(params, dtrain, num_boost_round=10)

# If no errors occurred, XGBoost is using the GPU
print("XGBoost is using the GPU.")



# CatBoost check
from catboost import CatBoostClassifier
# Initialize CatBoost model with GPU usage
model = CatBoostClassifier(task_type='GPU', iterations=10)

# Train on dummy data
model.fit([[0, 1], [1, 0], [0, 0], [1, 1]], [0, 1, 0, 1])

print("CatBoost successfully used the GPU.")

# FastAI check
from fastai.tabular.all import *

# Create a simple DataFrame
df = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': [1, 0, 1, 0, 1], 'y': [0, 1, 0, 1, 0]})

# Split the data
dls = TabularDataLoaders.from_df(df, y_names='y', cat_names=['a'], cont_names=['b'], procs=[Categorify, Normalize])

# Initialize a simple model
learn = tabular_learner(dls, metrics=accuracy)

# Check if FastAI is using GPU
if torch.cuda.is_available():
    learn.to_fp16()
    print("FastAI is using the GPU.")
else:
    print("FastAI is not using the GPU.")




PyTorch - CUDA available: True
XGBoost is using the GPU.
Learning rate set to 0.5
0:	learn: 0.6242674	total: 5.07ms	remaining: 45.7ms
1:	learn: 0.5917165	total: 5.76ms	remaining: 23.1ms
2:	learn: 0.5615844	total: 6.21ms	remaining: 14.5ms
3:	learn: 0.5336707	total: 6.67ms	remaining: 10ms
4:	learn: 0.4862918	total: 7.04ms	remaining: 7.04ms
5:	learn: 0.4450405	total: 7.39ms	remaining: 4.93ms
6:	learn: 0.4089812	total: 7.75ms	remaining: 3.32ms
7:	learn: 0.3773298	total: 8.1ms	remaining: 2.02ms
8:	learn: 0.3494304	total: 8.46ms	remaining: 939us
9:	learn: 0.3360929	total: 8.88ms	remaining: 0us
CatBoost successfully used the GPU.
FastAI is using the GPU.


In [2]:
# LightGBM check
import lightgbm as lgb
import numpy as np

# Create a simple dataset
data = np.array([[1, 2], [3, 4], [5, 6]])  # Convert to NumPy array
label = np.array([0, 1, 0])  # Convert to NumPy array
train_data = lgb.Dataset(data, label=label)

# Define parameters for LightGBM with GPU usage
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',

}

# Train the LightGBM model
gbm = lgb.train(params, train_data, num_boost_round=10)

# If no errors occurred, LightGBM is using the GPU
print("LightGBM is using the GPU.")

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 2
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 3, number of used features: 0
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because ther

In [3]:

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from autogluon.tabular import TabularPredictor, TabularDataset

## Evaluate the Model & save the preprocessor
import joblib
# Define the root directory
root_dir = '/workspace/Dataset/FSI/'

# Load the datasets
# train_df = pd.read_csv(root_dir + 'train_sample.csv')
# train_generated_df = pd.read_csv(root_dir + 'submission/filtered_data_3000.csv')
# test_df = pd.read_csv(root_dir + 'test.csv')


train_df = pd.read_csv(root_dir + 'train.csv')
train_generated_df = pd.read_csv(root_dir + 'generated_sample.csv')
test_df = pd.read_csv(root_dir + 'test.csv')

# Concatenate the training datasets
combined_train_df = pd.concat([train_df, train_generated_df], ignore_index=True)

# combined_train_df = train_generated_df

# Drop the 'ID' column from training data (but not 'Fraud_Type')
combined_train_df = combined_train_df.drop(columns=['ID'], errors='ignore')
print("combined_train_df after dropping ID:")
display(combined_train_df.head())  # Displays the first few rows after dropping 'ID'

test_df = test_df.drop(columns=['Fraud_Type'],errors='ignore')
print("test_df after dropping ID:")
display(test_df.head())  # Displays the first few rows after dropping 'ID'

# Check if val.csv exists; if not, split the combined_train_df
val_file_path = root_dir + 'train.csv'
if os.path.exists(val_file_path):
    val_df = pd.read_csv(val_file_path)
    val_df = val_df.drop(columns=['ID'], errors='ignore')  # Only drop 'ID', keep 'Fraud_Type' as the label
    print("val_df (loaded from train.csv):")
    display(val_df.head())  # Displays the first few rows of val_df
else:
    combined_train_df, val_df = train_test_split(combined_train_df, test_size=0.2, random_state=42)
    print("combined_train_df after splitting:")
    display(combined_train_df.head())  # Displays the first few rows of combined_train_df after splitting
    print("val_df (created by splitting):")
    display(val_df.head())  # Displays the first few rows of val_df created by splitting


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


combined_train_df after dropping ID:


,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date
0,1980,male,이상호,BJWQxd-WBASPLJ,2003-01-06 18:38:01,B,0,1,0,1,...,2003-01-22 23:38:48,2003-01-22 23:38:48,1,1,1,0,0,0,m,2003-01-22 23:38:48
1,1964,male,박상철,kurCwX-odPUXEt,2003-01-07 16:40:44,C,0,1,0,0,...,2003-01-21 21:29:08,2003-01-31 00:19:46,0,1,0,0,0,0,m,2003-01-19 21:29:08
2,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,0,0,...,2003-01-31 07:13:28,2003-01-31 07:13:28,0,0,1,1,1,0,m,2003-01-31 07:13:28
3,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,2003-01-31 11:49:56,2003-01-31 07:13:28,1,1,0,0,0,0,m,2003-01-31 07:13:28
4,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,2003-01-31 11:49:56,2003-01-31 07:13:28,1,0,0,1,1,0,m,2003-01-31 07:13:28


test_df after dropping ID:


,ID,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,...,Unused_terminal_status,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Transaction_resumed_date
0,TEST_000000,1960,female,주지아,DOMcBN-kRMFflJ,2003-01-07 10:59:08,E,1,0,0,...,1,2003-01-10 05:27:56,2003-01-08 05:27:56,0,1,1,0,0,0,2003-01-08 05:27:56
1,TEST_000001,1960,female,주지아,DOMcBN-kRMFflJ,2003-01-07 10:59:08,E,1,1,1,...,0,2003-01-11 21:29:50,2003-01-08 05:27:56,0,1,0,0,0,0,2003-01-08 05:27:56
2,TEST_000002,1951,male,김정수,pZrAvI-mhxfVyw,2003-01-06 18:10:55,B,1,1,1,...,0,2003-01-13 01:08:19,2003-01-13 01:08:19,1,0,0,2,2,0,2003-01-13 01:08:19
3,TEST_000003,1999,female,김현지,fVlbzX-wvugTpH,2003-01-08 05:28:53,B,0,1,1,...,1,2003-01-21 10:03:32,2003-01-26 13:49:24,0,1,1,0,0,0,2003-01-20 10:03:32
4,TEST_000004,1996,female,박은정,chYftA-AjVuXMW,2003-01-17 03:37:22,A,0,1,0,...,1,2003-01-28 19:04:19,2003-01-28 19:04:19,0,1,1,0,0,0,2003-01-28 19:04:19


val_df (loaded from train.csv):


,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date
0,1980,male,이상호,BJWQxd-WBASPLJ,2003-01-06 18:38:01,B,0,1,0,1,...,2003-01-22 23:38:48,2003-01-22 23:38:48,1,1,1,0,0,0,m,2003-01-22 23:38:48
1,1964,male,박상철,kurCwX-odPUXEt,2003-01-07 16:40:44,C,0,1,0,0,...,2003-01-21 21:29:08,2003-01-31 00:19:46,0,1,0,0,0,0,m,2003-01-19 21:29:08
2,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,0,0,...,2003-01-31 07:13:28,2003-01-31 07:13:28,0,0,1,1,1,0,m,2003-01-31 07:13:28
3,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,2003-01-31 11:49:56,2003-01-31 07:13:28,1,1,0,0,0,0,m,2003-01-31 07:13:28
4,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,2003-01-31 11:49:56,2003-01-31 07:13:28,1,0,0,1,1,0,m,2003-01-31 07:13:28


## Preprocess
Delete & create columns for better classification

In [4]:
df_list = [combined_train_df, val_df, test_df]

date_columns = [
    'Customer_registration_datetime',
    'Account_creation_datetime',
    'Transaction_Datetime',
    'Last_atm_transaction_datetime',
    'Last_bank_branch_transaction_datetime',
    'Transaction_resumed_date'
]

# Loop over each DataFrame in the list
for k in range(len(df_list)):
    df = df_list[k]
    
    # Convert 'Time_difference' to seconds
    df['Time_difference_seconds'] = pd.to_timedelta(df['Time_difference']).dt.total_seconds()
    
    # Convert the specified columns to datetime format
    for column in date_columns:
        df[column] = pd.to_datetime(df[column])
    
    # Create differences between the date columns
    for i in range(len(date_columns)):
        for j in range(i + 1, len(date_columns)):
            col_name = f'Diff_{date_columns[i]}_to_{date_columns[j]}'
            df[col_name] = (df[date_columns[j]] - df[date_columns[i]]).dt.days
    
    # Drop the original date columns and the 'Time_difference' column
    df.drop(columns=['Time_difference'] + date_columns, axis=1, inplace=True)
    
    # Drop additional unnecessary columns
    df.drop(columns=[
        'Customer_personal_identifier',
        'Customer_identification_number',
        'Account_account_number',
        'Error_Code', 
        'Type_General_Automatic',
        'IP_Address',
        'MAC_Address',
        'Location',
        'Recipient_Account_Number',
        'Another_Person_Account',
        'First_time_iOS_by_vulnerable_user'
    ], axis=1, inplace=True)
    
    # Update the DataFrame in the list (though this is redundant since df is a reference)
    df_list[k] = df
    
# Display the first few rows of each DataFrame
display(combined_train_df.head())
display(val_df.head())  
display(test_df.head())


,Customer_Birthyear,Customer_Gender,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,Customer_VPN_Indicator,...,Diff_Account_creation_datetime_to_Transaction_Datetime,Diff_Account_creation_datetime_to_Last_atm_transaction_datetime,Diff_Account_creation_datetime_to_Last_bank_branch_transaction_datetime,Diff_Account_creation_datetime_to_Transaction_resumed_date,Diff_Transaction_Datetime_to_Last_atm_transaction_datetime,Diff_Transaction_Datetime_to_Last_bank_branch_transaction_datetime,Diff_Transaction_Datetime_to_Transaction_resumed_date,Diff_Last_atm_transaction_datetime_to_Last_bank_branch_transaction_datetime,Diff_Last_atm_transaction_datetime_to_Transaction_resumed_date,Diff_Last_bank_branch_transaction_datetime_to_Transaction_resumed_date
0,1980,male,B,0,1,0,1,0,0,0,...,2,0,0,0,-3,-3,-3,0,0,0
1,1964,male,C,0,1,0,0,0,0,0,...,11,2,11,0,-10,-1,-12,9,-2,-12
2,1982,female,B,1,1,0,0,0,0,0,...,0,0,0,0,-1,-1,-1,0,0,0
3,1982,female,B,1,1,1,0,0,0,0,...,0,0,0,0,-1,-1,-1,-1,-1,0
4,1982,female,B,1,1,1,0,0,0,0,...,0,0,0,0,-1,-1,-1,-1,-1,0


,Customer_Birthyear,Customer_Gender,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,Customer_VPN_Indicator,...,Diff_Account_creation_datetime_to_Transaction_Datetime,Diff_Account_creation_datetime_to_Last_atm_transaction_datetime,Diff_Account_creation_datetime_to_Last_bank_branch_transaction_datetime,Diff_Account_creation_datetime_to_Transaction_resumed_date,Diff_Transaction_Datetime_to_Last_atm_transaction_datetime,Diff_Transaction_Datetime_to_Last_bank_branch_transaction_datetime,Diff_Transaction_Datetime_to_Transaction_resumed_date,Diff_Last_atm_transaction_datetime_to_Last_bank_branch_transaction_datetime,Diff_Last_atm_transaction_datetime_to_Transaction_resumed_date,Diff_Last_bank_branch_transaction_datetime_to_Transaction_resumed_date
0,1980,male,B,0,1,0,1,0,0,0,...,2,0,0,0,-3,-3,-3,0,0,0
1,1964,male,C,0,1,0,0,0,0,0,...,11,2,11,0,-10,-1,-12,9,-2,-12
2,1982,female,B,1,1,0,0,0,0,0,...,0,0,0,0,-1,-1,-1,0,0,0
3,1982,female,B,1,1,1,0,0,0,0,...,0,0,0,0,-1,-1,-1,-1,-1,0
4,1982,female,B,1,1,1,0,0,0,0,...,0,0,0,0,-1,-1,-1,-1,-1,0


,ID,Customer_Birthyear,Customer_Gender,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,...,Diff_Account_creation_datetime_to_Transaction_Datetime,Diff_Account_creation_datetime_to_Last_atm_transaction_datetime,Diff_Account_creation_datetime_to_Last_bank_branch_transaction_datetime,Diff_Account_creation_datetime_to_Transaction_resumed_date,Diff_Transaction_Datetime_to_Last_atm_transaction_datetime,Diff_Transaction_Datetime_to_Last_bank_branch_transaction_datetime,Diff_Transaction_Datetime_to_Transaction_resumed_date,Diff_Last_atm_transaction_datetime_to_Last_bank_branch_transaction_datetime,Diff_Last_atm_transaction_datetime_to_Transaction_resumed_date,Diff_Last_bank_branch_transaction_datetime_to_Transaction_resumed_date
0,TEST_000000,1960,female,E,1,0,0,0,1,0,...,3,2,0,0,-2,-4,-4,-2,-2,0
1,TEST_000001,1960,female,E,1,1,1,1,0,0,...,9,3,0,0,-6,-10,-10,-4,-4,0
2,TEST_000002,1951,male,B,1,1,1,1,0,0,...,6,0,0,0,-7,-7,-7,0,0,0
3,TEST_000003,1999,female,B,0,1,1,1,0,0,...,6,1,6,0,-6,-1,-7,5,-1,-7
4,TEST_000004,1996,female,A,0,1,0,0,1,0,...,0,0,0,0,-1,-1,-1,0,0,0


## Train the Model

We now specify the `Fraud_Type` as the target column and use AutoGluon's `TabularPredictor` to train the model with the `best_quality` preset.

In [5]:
# Specify the target column for classification
label = 'Fraud_Type'


# Initialize the TabularPredictor
predictor = TabularPredictor(label=label, eval_metric='accuracy').fit(
    train_data=TabularDataset(combined_train_df), 
    tuning_data=TabularDataset(val_df), 
    # presets='best_quality',# Options: 'best_quality', 'high_quality', 'good_quality', 'medium_quality'
)

leaderboard = predictor.leaderboard(silent=False)

No path specified. Models will be saved in: "AutogluonModels/ag-20240829_081249"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #129~20.04.1-Ubuntu SMP Wed Aug 7 13:07:13 UTC 2024
CPU Count:          20
Memory Avail:       20.45 GB / 31.15 GB (65.7%)
Disk Space Avail:   158.41 GB / 467.89 GB (33.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' :

## Evaluate the Model 

Let's evaluate the model on both the training and validation datasets to see how well it performs.

In [ ]:
# Evaluate performance on the training and validation datasets
train_performance = predictor.evaluate(combined_train_df)
val_performance = predictor.evaluate(val_df)

# Display performance metrics
print("Training performance metrics:")
print(train_performance)

print("\nValidation performance metrics:")
print(val_performance)


## Save the preprocessor used in Autogluon for future use

In [ ]:
# Access the feature generator from the predictor
preprocessor = predictor._learner.feature_generator

# Save the preprocessor using joblib
import joblib
joblib.dump(preprocessor, root_dir + 'preprocessor.pkl')

## Detailed Classification Report

We'll generate a detailed classification report for each fraud type using the validation dataset.

In [ ]:
# Make predictions on the validation set and calculate per-class scores
val_predictions = predictor.predict(val_df)
val_true = val_df[label]
classification_report_per_class = classification_report(val_true, val_predictions, output_dict=True)

# Display classification report for each fraud type
for fraud_type, metrics in classification_report_per_class.items():
    if isinstance(metrics, dict):  # Filter out 'accuracy' and other aggregate scores
        print(f"Fraud Type: {fraud_type}")
        for metric, score in metrics.items():
            print(f"  {metric}: {score}")
        print()

## Make Predictions on the Test Set

We now make predictions on the test set and prepare the submission file.

In [ ]:
# Make predictions on the test set
predictions = predictor.predict(test_df)

# Prepare the submission file using the ID column from the test data
submission = pd.DataFrame({
    'ID': test_df['ID'],  # Replace 'ID' with the actual ID column name in your test.csv
    'Fraud_Type': predictions
})

# Save the submission file
submission_file_path = root_dir + 'clf_submission.csv'
submission.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")

Syn_Submission Save

In [ ]:
train_original = pd.read_csv(root_dir + 'train.csv')
generated_original = pd.read_csv(root_dir + 'generated_data.csv')

generated_original['Predicted_Fraud_Type'] = predictor.predict(val_df)
probabilities = predictor.predict_proba(val_df)
generated_original['Confidence_Score'] = probabilities.max(axis=1)

correct_predictions_df = generated_original[generated_original['Fraud_Type'] == generated_original['Predicted_Fraud_Type']]

# Step 3: Sort by confidence score and select the top 1000 samples per Fraud_Type
sampled_df = correct_predictions_df.groupby('Fraud_Type').apply(lambda x: x.nlargest(1000, 'Confidence_Score')).reset_index(drop=True)

# Step 4: (Optional) Remove the 'Predicted_Fraud_Type' and 'Confidence_Score' columns if not needed
sampled_df = sampled_df.drop(columns=['Predicted_Fraud_Type', 'Confidence_Score'])

generated_original = generated_original.drop(columns=['Predicted_Fraud_Type', 'Confidence_Score'])


print(sampled_df)

In [ ]:
submission_format = pd.read_csv(root_dir + 'generated_data_submission.csv')
submission_dtypes = test_df.dtypes
sampled_df  = sampled_df[submission_format.columns]

# Convert the syn_submission.csv columns to match the train.csv data types
for column in submission_format.columns:
    if column =='ID' or column=="Fraud_Type":
        continue
    if sampled_df[column].dtype != submission_dtypes [column]:
        sampled_df[column] = sampled_df[column].astype(submission_dtypes [column])
        print(f"column '{column}' different!")


# Save the sampled DataFrame
sampled_df.to_csv(root_dir + 'syn_submission.csv', index=False)